In [1]:
import os
import numpy as np

import nibabel as nib
from nilearn import image, plotting, masking
from nilearn.maskers import NiftiMasker

import imageio.v3 as iio

In [2]:
fmri_file = 'data/sub-01_ses-V1_task-S1_run-03_space-MNI152NLin2009cAsym_res-2_desc-denoisedSmoothed_bold.nii.gz'
fmri_img = nib.load(fmri_file)

In [6]:
# Make brain mask and display
masker = NiftiMasker(mask_strategy="whole-brain-template", mask_args=dict(opening=2, threshold=0.2), smoothing_fwhm=8, standardize='zscore_sample')
masker.fit(fmri_file)

report = masker.generate_report()
report

In [7]:
# Apply the masker to the data
masked_data = masker.transform(fmri_file)
masked_img = masker.inverse_transform(masked_data)

In [8]:
# Get the number of volumes
x, y, z, n_volumes = masked_img.shape
print(n_volumes)

569


In [9]:
# Get the min and max values
vmax = np.max(masked_img.get_fdata())
vmin = np.min(masked_img.get_fdata())

print(vmin, vmax)

-9.245349414765766 7.20252244942664


In [48]:
step = 1

In [49]:
# Create images
for i in range(0, n_volumes, step):
    volume = image.index_img(masked_img, i)
    display = plotting.plot_epi(volume, title=f't = {i}', display_mode='ortho', cut_coords=(0, 0, 0), colorbar=True, vmin=vmin, vmax=vmax)
    display.savefig(f'static_images/volume_{i}.png')
    display.close()

In [51]:
frames = np.stack([iio.imread(f'static_images/volume_{x}.png') for x in range(0, n_volumes, step)], axis=0)
iio.imwrite('movie.gif', frames, format='gif', fps=300)